In [19]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
from pulp import *
import pandas as pd

# **Data**

**Add data**

In [21]:
# data
feedstocks = ['Alkylate', 'CCG', 'SRG', 'Isopentane']
gas_types = ['A', 'B', 'C']

#feedstocks data
gallons_available = dict(zip(feedstocks, [140000, 130000, 140000, 110000]))
value_per_gallon = dict(zip(feedstocks, [4.50, 2.50, 2.25, 2.35]))
reid_vapor_pressure = dict(zip(feedstocks, [5, 8, 4, 20]))
octane_rating_low = dict(zip(feedstocks, [98, 87, 83, 101]))
octane_rating_high = dict(zip(feedstocks, [107, 93, 89, 108]))

#Gas data
gallons_required = dict(zip(gas_types, [120000, 130000, 120000]))
price_per_gallon = dict(zip(gas_types, [3.00, 3.50, 4.00]))
max_reid_pressure = dict(zip(gas_types, [7, 7, 7]))
min_octane_rating = dict(zip(gas_types, [90, 97, 100]))


#**Programming**
**Formulate Linear Programming (LP) Problem**

In [22]:
# problem definition
prob = LpProblem('GasolineBlendingProblem', LpMaximize)

In [23]:
#Decision Variables
variable = LpVariable.dicts("gasoline", (feedstocks, gas_types), 0, None, LpContinuous)

In [24]:
print(variable)

{'Alkylate': {'A': gasoline_Alkylate_A, 'B': gasoline_Alkylate_B, 'C': gasoline_Alkylate_C}, 'CCG': {'A': gasoline_CCG_A, 'B': gasoline_CCG_B, 'C': gasoline_CCG_C}, 'SRG': {'A': gasoline_SRG_A, 'B': gasoline_SRG_B, 'C': gasoline_SRG_C}, 'Isopentane': {'A': gasoline_Isopentane_A, 'B': gasoline_Isopentane_B, 'C': gasoline_Isopentane_C}}


In [25]:
#Leftover Calculation
Leftover = {f: gallons_available[f] - lpSum(variable[f]) for f in feedstocks}

In [26]:

#Selling gasoline Calculation
sell_gas= {g: lpSum(variable[f][g] for f in feedstocks) for g in gas_types}

**Objective Function**

In [27]:
#Objective function for Revenue
prob += lpSum(price_per_gallon[g] * sell_gas[g] for g in gas_types) + lpSum(value_per_gallon[f] * Leftover[f] for f in feedstocks), "Total Revenue"

**Constraints**

In [28]:
#ReidVapour Constraints
for g in gas_types:
    prob += (variable[f][g]* reid_vapor_pressure[f] for f in feedstocks) <= lpSum(variable[f][g]*max_reid_pressure[g] for f in feedstocks)

In [29]:
#Octane Rating Constraints
for g in gas_types:
    if g == 'A':
        prob += lpSum([variable[f][g] * octane_rating_low[f] for f in feedstocks]) >= lpSum(variable[f][g] * min_octane_rating[g] for f in feedstocks)
    else:
        prob += lpSum([variable[f][g] * octane_rating_high[f] for f in feedstocks]) >= lpSum(variable[f][g] * min_octane_rating[g] for f in feedstocks)

In [30]:
#Gas A produced is atleast as large as Gas B
prob += lpSum(variable[f]["A"] for f in feedstocks) >= lpSum(variable[f]["B"] for f in feedstocks)

In [31]:
#Total Gallons made greater then Required
for g in gas_types:
    prob += lpSum(variable[f][g] for f in feedstocks) >= gallons_required[g]

In [32]:
#Total Used should be less than Availability
for f in feedstocks:
    prob += lpSum(variable[f]) <= gallons_available[f]
    

**Run Optimization Algorithm**

In [33]:
prob.solve()

1

**Get Status & Results**

In [34]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [35]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

gasoline_Alkylate_A = 37131.064
gasoline_Alkylate_B = 34312.268
gasoline_Alkylate_C = 68556.668
gasoline_CCG_A = 87887.023
gasoline_CCG_B = 0.0
gasoline_CCG_C = 0.0
gasoline_Isopentane_A = 82.552639
gasoline_Isopentane_B = 22230.483
gasoline_Isopentane_C = 24772.578
gasoline_SRG_A = 4899.36
gasoline_SRG_B = 73457.249
gasoline_SRG_C = 61643.391


In [36]:
print("Total revenue = ", value(prob.objective))


Total revenue =  1718021.7973653502


In [37]:
a = [{'name':name,'shadow price':g.pi,'slack': g.slack} for name, g in prob.constraints.items()]

print(pd.DataFrame(a))

    name  shadow price         slack
0    _C1     -0.272305 -0.000000e+00
1    _C2     -0.240019 -0.000000e+00
2    _C3     -0.240019 -0.000000e+00
3    _C4     -0.222690 -0.000000e+00
4    _C5     -0.183782  7.909289e-12
5    _C6     -0.183782  7.163692e-12
6    _C7     -0.440377 -0.000000e+00
7    _C8     -0.000000 -1.000000e+04
8    _C9     -0.389032 -0.000000e+00
9   _C10     -0.000000 -3.497264e+04
10  _C11      1.266511 -0.000000e+00
11  _C12     -0.000000  4.211298e+04
12  _C13      0.448460 -0.000000e+00
13  _C14     -0.000000  6.291439e+04
